In [2]:
import pandas as pd
import numpy as np
import os
from scipy.stats import zscore, rankdata
from heapq import nlargest
import random
import copy
import math

In [3]:
# head to head format
# use steamers projections for user
cwd = os.getcwd()
c_path = os.path.join(cwd, "Data", "Catchers 2020 Projections.csv")
first_path = os.path.join(cwd, "Data", "First Basemen 2020 Projections.csv")
second_path = os.path.join(cwd, "Data", "Second Basemen 2020 Projections.csv")
short_path = os.path.join(cwd, "Data", "Shortstop 2020 Projections.csv")
third_path = os.path.join(cwd, "Data", "Third Basemen 2020 Projections.csv")
outfield_path = os.path.join(cwd, "Data", "Outfielders 2020 Projections.csv")
dh_path = os.path.join(cwd, "Data", "Designated Hitters 2020 Projections.csv")
pitchers_path = os.path.join(cwd, "Data", "Pitchers 2020 Projections.csv")
espn_path = os.path.join(cwd, "Data", "ESPN Top 300 Players (Cockcroft).csv")
test_path = os.path.join(cwd, "Data", "FantasyPros 2020 Projections Hitters.csv")

# read csvs into dataframes
catchers_proj = pd.read_csv(c_path)
first_proj = pd.read_csv(first_path)
second_proj = pd.read_csv(second_path)
ss_proj = pd.read_csv(short_path)
third_proj = pd.read_csv(third_path)
of_proj = pd.read_csv(outfield_path)
dh_proj = pd.read_csv(dh_path)
# pitchers_proj = pd.read_csv(pitchers_path)
opp_proj = pd.read_csv(espn_path)
test_data = pd.read_csv(test_path)

# test_data.rename(columns = {'K':'SO'}, inplace=True)

# print(test_data[test_data.Player.str.contains('Maldonado')])

# create a dictionary of positional databases
pos_players = {'C': catchers_proj, '1B': first_proj, '2B': second_proj, '3B': third_proj, 'SS': ss_proj\
               , 'OF': of_proj}

pos_cat = {'R':1, 'H':1, 'RBI':1, 'BB':1, 'SO':-1, 'SB':1}

# multiply the pitchers overall stats for each by their point totals, 
pitchers_cat = {'IP':3, 'H':-1, 'ER':-2, 'BB':-1, 'K':1, 'W':5, 'L':-5, 'SV':5}

# create a dictionary of the spots that each team is drafting
glob_spots = {'C': 1, '1B': 2, '2B': 2, '3B': 2, 'SS': 2, 'OF': 5, 'UTIL': 1}

In [4]:
test_data = test_data.rename(columns={"Player":"Name", "Positions":"Pos"})
test_data['Pos'] = test_data['Pos'].str[:2]
test_data = test_data[test_data.Name.notna()]

print(test_data)

test_data.loc[57, 'Name'] = 'Nicholas Castellanos'
test_data.loc[132, 'Name'] = 'Giovanny Urshela'
test_data.loc[29, 'Name'] = 'Peter Alonso'
test_data.loc[136, 'Name'] = 'Yasiel Puig'

                      Name Team Pos     AB      R    HR    RBI    SB    AVG  \
0               Mike Trout  LAA  CF  444.0  103.0  38.0   94.0  14.0  0.308   
1             Alex Bregman  HOU  3B  477.0   94.0  30.0   92.0   7.0  0.295   
2           Cody Bellinger  LAD  1B  478.0   94.0  36.0   96.0  11.0  0.293   
3            Nolan Arenado  COL  3B  507.0   88.0  35.0  103.0   2.0  0.304   
4             Mookie Betts  LAD  CF  505.0  104.0  27.0   77.0  17.0  0.297   
..                     ...  ...  ..    ...    ...   ...    ...   ...    ...   
636        Troy Stokes Jr.  DET  LF    5.0    1.0   0.0    1.0   0.0  0.180   
637             Joe Hudson  SEA   C   16.0    2.0   0.0    2.0   0.0  0.188   
638  Luis Alexander Basabe  CWS  CF   18.0    2.0   0.0    2.0   1.0  0.233   
639              Skye Bolt  OAK  CF    9.0    1.0   0.0    1.0   0.0  0.222   
640           Sam Haggerty  SEA  2B    8.0    1.0   0.0    0.0   0.0  0.248   

       OBP      H    2B   3B     BB     SO    SLG  

In [5]:
print(test_data[test_data.Name.str.contains('Puig')])

            Name Team Pos     AB     R    HR   RBI    SB    AVG    OBP      H  \
136  Yasiel Puig  NaN  RF  378.0  55.0  18.0  59.0  11.0  0.271  0.333  103.0   

       2B   3B    BB    SO    SLG    OPS    PTS  
136  19.0  2.0  35.0  89.0  0.471  0.804  249.0  


In [6]:
# data cleaning, fix the opponent database to match the user's
opp_proj = opp_proj.rename(columns={"Player":"Name", "Elig. Pos.":"Pos", "Pos.\nRank":"Rank"})

opp_proj['Pos'] = opp_proj['Pos'].str[:2]

opp_proj = opp_proj[opp_proj.Pos != 'SP']

opp_proj = opp_proj[opp_proj.Pos != 'RP']

opp_proj.loc[28, 'Name'] = 'Peter Alonso'

opp_proj.loc[220, 'Name'] = 'Giovanny Urshela'

# Grandal was both C and 1B
opp_proj.loc[141, 'Pos'] = 'C'

# Matt Carpenter is 1B in Fangraphs but 3B in ESPN, move him to 1B
opp_proj.loc[219, 'Pos'] = '1B'

# Fangraphs for some reason does not have Nick Madrigal or Shogo Akiyama in Steamers, ZIPS, or FGDepth Charts
opp_proj = opp_proj[opp_proj.Name != 'Nick Madrigal']

opp_proj = opp_proj[opp_proj.Name != 'Shogo Akiyama']

# print (opp_proj[opp_proj.Name.str.contains('Harold')])

opp_proj = opp_proj[opp_proj.Pos != 'DH']

print(opp_proj)

    Rank               Name Team Pos  Rank
0      1         Mike Trout  LAA  OF   OF1
1      2   Ronald Acuna Jr.  ATL  OF   OF2
2      3   Christian Yelich  MIL  OF   OF3
3      4     Cody Bellinger  LAD  OF   OF4
5      6       Mookie Betts  LAD  OF   OF5
..   ...                ...  ...  ..   ...
291  292       Justin Smoak  MIL  1B  1B29
293  294        Travis Shaw  TOR  3B  3B33
296  297          Jon Berti  MIA  SS  SS28
297  298        Eric Thames  WSH  1B  1B30
299  300  Anthony Santander  BAL  OF  OF82

[176 rows x 5 columns]


In [9]:
x = combine_df(pos_players, 80, pos_cat)

print (x[x.Name.str.contains('Choo')])
print(of_proj.head(1))

             Name Pos  Average Value
48  Shin-Soo Choo  OF       0.757896
         Name    Team    G   PA   AB    H  2B  3B  HR    R  ...  wRC+  BsR  \
0  Mike Trout  Angels  150  679  530  157  30   3  44  127  ...   173  3.2   

   Fld  -1.2   Off  Def  WAR  -1.3  ADP  playerid  
0 -1.7   NaN  66.9  0.6  8.9   NaN  2.3     10155  

[1 rows x 33 columns]


In [8]:
# for an input position and dataframe, find the average value of each player
def avg_val_pos(pos, dataframe, categories):
    qual_player = dataframe
    qual_col = qual_player[list(categories.keys())]

    norm_col = qual_col.apply(zscore)
    qual_player['Pos'] = pos
    qual_player['Average Value'] = norm_col.mean(axis=1)
    return qual_player[['Name','Pos', 'Average Value']]

# create a dataframe of the combined positons with each player's average value
def combine_df(dict_dataset, game_min, categories):
    pos_val_df = pd.DataFrame(columns=['Name','Pos', 'Average Value'])
    for key, value in dict_dataset.items():
        value = value[value.G > game_min]
        value = value.reset_index()
        value.drop('index', axis=1, inplace=True)
        val = (avg_val_pos(key, value, categories))
        pos_val_df = pos_val_df.append(val, ignore_index=True)
    pos_val_df = pos_val_df.sort_values('Average Value', ascending = False)
    no_dups_table = pos_val_df.drop_duplicates(subset='Name', keep='first')
    no_dups_table = no_dups_table.reset_index()
    no_dups_table.drop('index', axis=1, inplace=True)
    return no_dups_table

combine_df(pos_players, 100, pos_cat)

,Name,Pos,Average Value
0,Mike Trout,OF,1.787806
1,Ronald Acuna Jr.,OF,1.679033
2,Cody Bellinger,1B,1.616789
3,Christian Yelich,OF,1.527615
4,Bryce Harper,OF,1.323676
...,...,...,...
202,Alex Dickerson,OF,-1.210913
203,Ryan O'Hearn,1B,-1.252340
204,Colin Moran,3B,-1.260921
205,Adam Haseley,OF,-1.391497


In [10]:
def sort_normed_df(norm_df):
    norm_df = norm_df.sort_values('Average Value', ascending=False)
    norm_df = norm_df.reset_index()
    norm_df.drop('index', axis=1, inplace=True)
    return norm_df

In [96]:
# given a drafted player, redistribute the user's database. if the user drafted the player, then need to update
# their team's spots and the database with a discount factor
def redist_df(dataframe, spots, player, discount=0.8, drafted=False):
    row_index = dataframe[dataframe.Name==player].index
    row = dataframe.iloc[row_index]
    dataframe.drop(row_index, inplace=True)
    drafted_pos = (row.loc[row_index, 'Pos'].values)[0]
    normed_df = dataframe[['Name', 'Pos']]
    normed_df['Average Value'] = dataframe[['Average Value']].apply(zscore)
    if drafted:
        if drafted_pos in spots:
            if spots[drafted_pos] > 0:
                data = spots[drafted_pos]
#                 discount_factor = 1 - (1 / data)
                discount_factor = discount
            elif spots['UTIL'] > 0:
                spots[drafted_pos] += 1
#                 discount_factor = 1 - (1 / spots['UTIL'])
                discount_factor = 1
                spots['UTIL'] -= 1
            else:
                normed_df = remove_pos(normed_df, drafted_pos)
                discount_factor = 0
        else:
            return 'Unknown position'
        for index, row in normed_df.iterrows():
            if normed_df.loc[index, 'Pos'] == drafted_pos:
                if discount_factor == 0:
                    normed_df.loc[index, 'Average Value'] = 0
                elif normed_df.loc[index, 'Average Value'] >= 0:
                    normed_df.loc[index, 'Average Value'] = normed_df.loc[index, 'Average Value'] * discount_factor
                else:
                    normed_df.loc[index, 'Average Value'] = normed_df.loc[index, 'Average Value'] * (1 / discount_factor)
        return sort_normed_df(normed_df)
    return sort_normed_df(normed_df)

my_dataset = combine_df(pos_players, 110, pos_cat)
cell_spots = glob_spots
first_pick = redist_df(my_dataset, cell_spots, 'Mike Trout', 0.8, True)
print(first_pick)
second_pick = redist_df(first_pick, cell_spots, 'Cody Bellinger', 0.8, True)
print(second_pick)
third_pick = redist_df(second_pick, cell_spots, 'Yasmani Grandal', 0.8, True)
print(third_pick)
fourth_pick = redist_df(third_pick, cell_spots, 'Miguel Rojas', 0.8, True)
print(fourth_pick)
fifth_pick = redist_df(fourth_pick, cell_spots, 'Matt Carpenter')
print(fifth_pick)
print(cell_spots)

                 Name Pos  Average Value
0      Cody Bellinger  1B       2.109988
1        Trevor Story  SS       1.956626
2    Ronald Acuna Jr.  OF       1.898835
3        Jose Ramirez  3B       1.846423
4        Yoan Moncada  2B       1.750358
..                ...  ..            ...
177   Kevin Kiermaier  OF      -2.015378
178     Jason Heyward  OF      -2.021759
179   Marwin Gonzalez  OF      -2.021903
180      Miguel Rojas  SS      -2.146328
181        Kyle Lewis  OF      -2.177956

[182 rows x 3 columns]
                 Name Pos  Average Value
0        Trevor Story  SS       2.024254
1    Ronald Acuna Jr.  OF       1.966812
2        Jose Ramirez  3B       1.914717
3        Yoan Moncada  2B       1.819232
4    Christian Yelich  OF       1.787826
..                ...  ..            ...
176   Kevin Kiermaier  OF      -1.923778
177     Jason Heyward  OF      -1.930120
178   Marwin Gonzalez  OF      -1.930263
179      Miguel Rojas  SS      -2.053938
180        Kyle Lewis  OF      -2

In [12]:
# create a random draft order given the number of teams and number of spots for each position
def create_draft_order(num_teams, num_spots):
    draft_pos = list(range(1,num_teams+1))
    draft_order = random.sample(draft_pos, num_teams) # randomize the draft order 
    x = 1
    
    all_teams_spots = {}
    
    # copy the number of spots necessary for each team
    while x < 11:
        all_teams_spots['team{0}'.format(x)]= copy.deepcopy(num_spots)
        all_teams_spots['team{0}'.format(x)].update({'Draft Order' : draft_order[x - 1]})
        x += 1
    return draft_order, all_teams_spots

# pick the top player in the database and return their name and position
def pick_player(df, user=True, spots=None):
    if not user:
        for index, player in df.iterrows():
            if spots[player.Pos] > 0:
                return player.Name, player.Pos
    highest_player = df.iloc[0]
    return highest_player.Name, highest_player.Pos
        
# create_draft_order(10, spots)

def remove_pos(df, pos):
    return df[df.Pos != pos]

def remove_draftee(df, draftee):
    df = df[df.Name!=draftee]
    df = df.reset_index()
    df.drop('index', axis=1, inplace=True)
    return df

data = combine_df(pos_players, 80, pos_cat)
data = remove_pos(data, 'OF')
print (data)

                 Name Pos  Average Value
0     Yasmani Grandal   C       1.971312
2       J.T. Realmuto   C       1.888429
5      Cody Bellinger  1B       1.616607
7        Jose Ramirez  3B       1.414325
8        Trevor Story  SS       1.335255
..                ...  ..            ...
246       Austin Nola  1B      -1.105592
250      Miguel Rojas  SS      -1.258153
251   Tommy La Stella  3B      -1.276820
254     Johan Camargo  3B      -1.363645
255  Ryan Mountcastle  SS      -1.643133

[150 rows x 3 columns]


In [138]:
def round_selection(round_order, val_df, opp_df, teams, all_teams_spots, discount):
    no_more_picks = False
    for team_num in round_order:
        if opp_df.empty:
            no_more_picks = True
            return val_df, opp_df, teams, all_teams_spots, no_more_picks
        if team_num == 1:
            draftee, draftee_pos = pick_player(val_df)
            print(draftee)
            print(val_df)
            all_teams_spots['team{0}'.format(team_num)][draftee_pos] -= 1
            val_df = redist_df(val_df, all_teams_spots['team{0}'.format(team_num)], draftee, discount, True)
            print(val_df)
        else:
            draftee, draftee_pos = pick_player(opp_df, False, all_teams_spots['team{0}'.format(team_num)])
            all_teams_spots['team{0}'.format(team_num)][draftee_pos] -= 1
            if draftee in val_df.Name.values:
                val_df = redist_df(val_df, all_teams_spots['team{0}'.format(team_num)], draftee, discount)
        opp_df = remove_draftee(opp_df, draftee)
        teams[team_num].append(draftee)
    return val_df, opp_df, teams, all_teams_spots, no_more_picks

In [139]:
# def test(dict_of_datasets, num_teams, spots, opp_dataset=None):

def test(user_df, num_teams, spots, opp_dataset, discount):
    
    rounds = sum(spots.values()) + 1
    
    teams = dict((el,[]) for el in range(1, num_teams+1))
    
    forward_order, all_teams_spots = create_draft_order(num_teams, spots)
    
    first_drafter = forward_order.index(1) + 1
        
    # in order to keep track of who is drafting, need to substract a round each time the person with the 
    # first pick drafts, since it is snake draft and they will have the last pick of the draft
    
    reverse_order = [element for element in reversed(forward_order)]
    round_num = 1
    no_more_picks = opp_dataset.empty
    while round_num < rounds and not no_more_picks:
        if round_num % 2 == 1:
            round_order = forward_order
            user_df, opp_dataset, teams, all_teams_spots, no_more_picks = round_selection(round_order, user_df, opp_dataset, teams, all_teams_spots, discount)
        else:
            round_order = reverse_order
            user_df, opp_dataset, teams, all_teams_spots, no_more_picks = round_selection(round_order, user_df, opp_dataset, teams, all_teams_spots, discount)   
        round_num += 1
    return teams
                

opp_df = copy.copy(opp_proj)
my_data = combine_df(pos_players, 100, pos_cat)
test(my_data, 10, glob_spots, opp_df, 0.8)

Cody Bellinger
               Name Pos  Average Value
0    Cody Bellinger  1B       2.385333
1      Bryce Harper  OF       1.957393
2      Trevor Story  SS       1.944760
3      Jose Ramirez  3B       1.943504
4      Yoan Moncada  3B       1.756258
..              ...  ..            ...
199  Alex Dickerson  OF      -1.743048
200    Ryan O'Hearn  1B      -1.803530
201     Colin Moran  3B      -1.816058
202    Adam Haseley  OF      -2.006697
203    Miguel Rojas  SS      -2.040383

[204 rows x 3 columns]
              Name Pos  Average Value
0     Bryce Harper  OF       1.992433
1     Trevor Story  SS       1.979649
2     Jose Ramirez  3B       1.978379
3     Yoan Moncada  3B       1.788918
4      Trea Turner  SS       1.697176
..             ...  ..            ...
198      Luke Voit  1B      -1.890357
199   Adam Haseley  OF      -2.018540
200   Miguel Rojas  SS      -2.052625
201  Albert Pujols  1B      -2.080039
202   Ryan O'Hearn  1B      -2.266213

[203 rows x 3 columns]
Jose Ramirez


Shin-Soo Choo
                Name Pos  Average Value
0      Shin-Soo Choo  OF       2.090373
1   Andrew McCutchen  OF       1.842603
2       Brandon Lowe  2B       1.770619
3        Khris Davis  OF       1.717145
4        Paul DeJong  SS       1.682122
..               ...  ..            ...
72      Jesse Winker  OF      -1.706147
73    Ender Inciarte  OF      -1.888025
74    Alex Dickerson  OF      -1.914926
75      Miguel Rojas  SS      -2.276879
76      Adam Haseley  OF      -2.397268

[77 rows x 3 columns]
                Name Pos  Average Value
0       Brandon Lowe  2B       1.840091
1        Paul DeJong  SS       1.749528
2       Niko Goodrum  2B       1.572041
3   Andrew McCutchen  OF       1.531004
4        Khris Davis  OF       1.428295
..               ...  ..            ...
71      Jesse Winker  OF      -2.147276
72      Miguel Rojas  SS      -2.301873
73    Ender Inciarte  OF      -2.379929
74    Alex Dickerson  OF      -2.414341
75      Adam Haseley  OF      -3.031340

[7

Khris Davis
                  Name Pos  Average Value
0          Khris Davis  OF       1.525261
1     Mike Yastrzemski  OF       1.207624
2        Danny Santana  OF       1.085050
3    Anthony Santander  OF       0.982888
4    Teoscar Hernandez  OF       0.944309
5      Gregory Polanco  OF       0.939894
6           Mark Canha  OF       0.878864
7        Brandon Nimmo  OF       0.869056
8       Randal Grichuk  OF       0.823128
9        David Peralta  OF       0.748246
10      Hunter Renfroe  OF       0.730352
11        Kole Calhoun  OF       0.672387
12  Jackie Bradley Jr.  OF       0.664764
13      Cameron Maybin  OF       0.460720
14       Brett Gardner  OF       0.445962
15       Trent Grisham  OF       0.373878
16       Dexter Fowler  OF       0.370322
17     Corey Dickerson  OF       0.344448
18         Austin Hays  OF       0.297656
19        Alex Verdugo  OF       0.273242
20         Nick Senzel  OF       0.219728
21        JaCoby Jones  OF       0.115441
22        Nomar Mazara

{1: ['Cody Bellinger',
  'Jose Ramirez',
  'Aaron Judge',
  'J.T. Realmuto',
  'Adalberto Mondesi',
  'Yasmani Grandal',
  'Josh Donaldson',
  'Joey Votto',
  'Jorge Polanco',
  'Shin-Soo Choo',
  'Brandon Lowe',
  'Andrew McCutchen',
  'J.P. Crawford',
  'Khris Davis',
  'Mike Yastrzemski'],
 2: ['Mike Trout',
  'Jose Altuve',
  'Charlie Blackmon',
  'Jonathan Villar',
  'Matt Olson',
  'Kyle Schwarber',
  'Andrew Benintendi',
  'Adam Eaton',
  'Eduardo Escobar',
  'Mitch Garver',
  'C.J. Cron',
  'Didi Gregorius',
  'Kevin Newman',
  'Hunter Dozier',
  'Danny Santana'],
 3: ['Trea Turner',
  'Freddie Freeman',
  'Ozzie Albies',
  'Kris Bryant',
  'Whit Merrifield',
  'Matt Chapman',
  'Giancarlo Stanton',
  'Corey Seager',
  'Edwin Encarnacion',
  'Joc Pederson',
  'Bryan Reynolds',
  'Willie Calhoun',
  'J.D. Davis',
  'Christian Vazquez',
  'Nomar Mazara'],
 4: ['Nolan Arenado',
  'Trevor Story',
  'Manny Machado',
  'Anthony Rizzo',
  'Marcus Semien',
  'Michael Conforto',
  'Gary

In [140]:
opp_df = copy.copy(opp_proj)
my_data = combine_df(pos_players, 98, pos_cat)
test(my_data, 10, glob_spots, opp_df, 1.0)

Bryce Harper
                Name Pos  Average Value
0       Bryce Harper  OF       2.065466
1       Jose Ramirez  3B       2.010411
2       Trevor Story  SS       1.977486
3    Yasmani Grandal   C       1.895521
4      J.T. Realmuto   C       1.854801
..               ...  ..            ...
208      Austin Nola  1B      -1.774973
209         Rio Ruiz  3B      -1.777427
210     Adam Haseley  OF      -1.826338
211    Lewis Brinson  OF      -1.835315
212     Miguel Rojas  SS      -2.009885

[213 rows x 3 columns]
                Name Pos  Average Value
0       Jose Ramirez  3B       2.035995
1       Trevor Story  SS       2.002813
2    Yasmani Grandal   C       1.920205
3      J.T. Realmuto   C       1.879165
4       Yoan Moncada  3B       1.851069
..               ...  ..            ...
207      Austin Nola  1B      -1.779073
208         Rio Ruiz  3B      -1.781547
209     Adam Haseley  OF      -1.830841
210    Lewis Brinson  OF      -1.839888
211     Miguel Rojas  SS      -2.015827

[2

                Name Pos  Average Value
0   Andrew McCutchen  OF       2.207598
1       Brandon Lowe  2B       2.045175
2        Paul DeJong  SS       2.040676
3        Khris Davis  OF       2.033078
4       Justin Upton  OF       1.813519
..               ...  ..            ...
75  Christin Stewart  OF      -1.667697
76       Jake Fraley  OF      -1.675087
77      Adam Haseley  OF      -1.892320
78     Lewis Brinson  OF      -1.906678
79      Miguel Rojas  SS      -2.185877

[80 rows x 3 columns]
Andrew McCutchen
                Name Pos  Average Value
0   Andrew McCutchen  OF       2.357726
1        Paul DeJong  SS       2.185247
2        Khris Davis  OF       2.177396
3       Justin Upton  OF       1.950529
4       Niko Goodrum  2B       1.819986
..               ...  ..            ...
70  Christin Stewart  OF      -1.646573
71       Jake Fraley  OF      -1.654209
72      Adam Haseley  OF      -1.878673
73     Lewis Brinson  OF      -1.893509
74      Miguel Rojas  SS      -2.182003


{1: ['Bryce Harper',
  'Jose Ramirez',
  'Yasmani Grandal',
  'J.T. Realmuto',
  'Yoan Moncada',
  'Adalberto Mondesi',
  'Joey Votto',
  'Carlos Santana',
  'Shin-Soo Choo',
  'Jorge Polanco',
  'Andrew McCutchen',
  'Khris Davis',
  'Niko Goodrum',
  'Mike Yastrzemski',
  'J.P. Crawford'],
 2: ['Mike Trout',
  'Jose Altuve',
  'Charlie Blackmon',
  'Eddie Rosario',
  'Jonathan Villar',
  'Vladimir Guerrero Jr.',
  'Kyle Schwarber',
  'Oscar Mercado',
  'Eduardo Escobar',
  'Jean Segura',
  'Mitch Garver',
  'Luke Voit',
  'Kevin Newman',
  'Renato Nunez',
  'Yasiel Puig'],
 3: ['Ronald Acuna Jr.',
  'Javier Baez',
  'Peter Alonso',
  'Paul Goldschmidt',
  'Tommy Pham',
  'DJ LeMahieu',
  'Max Muncy',
  'Franmil Reyes',
  'Adam Eaton',
  'Mallex Smith',
  'Willson Contreras',
  'Didi Gregorius',
  'Tommy Edman',
  'Hunter Dozier',
  'Danny Santana'],
 4: ['Christian Yelich',
  'Starling Marte',
  'George Springer',
  'Aaron Judge',
  'Matt Olson',
  'Victor Robles',
  'Josh Donaldson'

In [16]:
my_data = combine_df(pos_players, 100, pos_cat)
teams = test(my_data, 10, glob_spots, opp_proj)
print(teams)

{1: ['Cody Bellinger', 'Yoan Moncada', 'Aaron Judge', 'J.T. Realmuto', 'Adalberto Mondesi', 'Jorge Soler', 'Jorge Polanco', 'Shin-Soo Choo', 'Yuli Gurriel', 'David Fletcher', 'Andrew McCutchen', 'Roberto Perez', 'Luis Urias', 'Kevin Newman', 'Gregory Polanco'], 2: ['Ronald Acuna Jr.', 'Starling Marte', 'Charlie Blackmon', 'Paul Goldschmidt', 'Matt Olson', 'DJ LeMahieu', 'Max Muncy', 'Oscar Mercado', 'David Dahl', 'Brian Anderson', 'Willson Contreras', 'Yandy Diaz', 'Didi Gregorius', 'Dansby Swanson', 'Avisail Garcia'], 3: ['Mookie Betts', 'Anthony Rendon', 'George Springer', 'Josh Bell', 'Keston Hiura', 'Bo Bichette', 'Josh Donaldson', 'Franmil Reyes', 'Byron Buxton', 'Amed Rosario', 'Mallex Smith', 'Wilson Ramos', 'Daniel Murphy', 'Starlin Castro', 'Mark Canha'], 4: ['Alex Bregman', 'J.D. Martinez', 'Austin Meadows', 'Kris Bryant', 'Jose Abreu', 'Carlos Correa', 'Michael Brantley', 'Corey Seager', 'Gavin Lux', 'Lourdes Gurriel Jr.', 'Scott Kingery', 'Salvador Perez', 'Michael Chavis',

In [92]:
def find_category_stats(team_dict, categories, compare_data):
    
#     all_data_list = []
#     for pos, data in data_dict.items():
#         all_data_list.append(data)
        
    team_totals = dict((el,0) for el in categories)
    
    totals = {}

    for x in range(1, len(team_dict.keys()) + 1):
        totals[x] = copy.copy(team_totals)
    
#     all_data = pd.concat(all_data_list)
    num_players = 0
    for team, players in team_dict.items():
        num_players = len(players)
        for player in players:
            player_data = compare_data.loc[compare_data.Name==player]
            for stat, value in team_totals.items():
                totals[team][stat] += player_data.iloc[0][str(stat)]
#                 print(player_data)
#                 print(player_data.iloc[0][str(stat)])
                
    return totals

all_team_stats_dict = find_category_stats(teams, pos_cat, test_data)
print(all_team_stats_dict)

{1: {'R': 989.0, 'H': 1761.0, 'RBI': 955.0, 'BB': 728.0, 'SO': 1583.0, 'SB': 124.0}, 2: {'R': 1033.0, 'H': 1838.0, 'RBI': 1065.0, 'BB': 752.0, 'SO': 1470.0, 'SB': 65.0}, 3: {'R': 1048.0, 'H': 1857.0, 'RBI': 1073.0, 'BB': 688.0, 'SO': 1627.0, 'SB': 95.0}, 4: {'R': 1065.0, 'H': 1841.0, 'RBI': 1069.0, 'BB': 750.0, 'SO': 1783.0, 'SB': 102.0}, 5: {'R': 1076.0, 'H': 1930.0, 'RBI': 965.0, 'BB': 653.0, 'SO': 1685.0, 'SB': 180.0}, 6: {'R': 1079.0, 'H': 1890.0, 'RBI': 1028.0, 'BB': 729.0, 'SO': 1607.0, 'SB': 115.0}, 7: {'R': 1053.0, 'H': 1850.0, 'RBI': 1026.0, 'BB': 693.0, 'SO': 1630.0, 'SB': 136.0}, 8: {'R': 1059.0, 'H': 1870.0, 'RBI': 1070.0, 'BB': 690.0, 'SO': 1618.0, 'SB': 111.0}, 9: {'R': 1023.0, 'H': 1820.0, 'RBI': 1043.0, 'BB': 626.0, 'SO': 1677.0, 'SB': 137.0}, 10: {'R': 1097.0, 'H': 1907.0, 'RBI': 1051.0, 'BB': 721.0, 'SO': 1678.0, 'SB': 111.0}}


In [95]:
def sep_by_cat(teams_stats_dict, categories):
    cats = dict((el,[]) for el in range(len(categories)))
    for team, stats in teams_stats_dict.items():
        x = 0
        for stat, num in stats.items():
            cats[x].append(num)
            x += 1
    return cats
    
def create_rankings(teams_stats_dict, cat_lists):
    rankings = []
    total = []
    team_sum = dict((el,[]) for el in range(len(teams_stats_dict.values())))
    for key, value in cat_lists.items():
        rankings.append(list(rankdata(value).astype(int)))
    
    for stats in rankings:
        for y in range(len(stats)):
            team_sum[y].append(stats[y])
    for ranks in team_sum.values():
        total.append(sum(ranks))
    return total

def rank_teams(teams_stats_dict, categories):
    cats = sep_by_cat(teams_stats_dict, categories)
    final_points = create_rankings(teams_stats_dict, cats)
    return final_points
    

my_data = combine_df(pos_players, 100, categories)
teams = test(my_data, 10, glob_spots, opp_df)
all_team_stats_dict = find_category_stats(teams, categories, test_data)
print(all_team_stats_dict)
rank_teams(all_team_stats_dict, categories)

AttributeError: 'list' object has no attribute 'keys'

In [48]:
from scipy.stats import rankdata

def absolute_ranking(num_teams, cat_points):
    for cat in cat_points:
        rev_ranks = rankdata(cat, 'max')
        ranks = [num_teams + 1 - el for el in rev_ranks]
        print(ranks)

In [133]:
import scipy.stats as st

def run(categories, games_min, spots, num_teams, trials, positional_df, test_data, opp_df, discount=0.9, confidence=0.95):
    user_df = combine_df(positional_df, games_min, categories)
    average_points = []
    for x in range(trials):
        user_df_copy = copy.copy(user_df)
        teams = test(user_df_copy, num_teams, spots, opp_df, discount)
        all_teams_stats_dict = find_category_stats(teams, categories, test_data)
        summed_cat = rank_teams(all_teams_stats_dict, categories)
        inter = sep_by_cat(all_teams_stats_dict, categories)
        average_points.append(summed_cat)
    mean, sem, m = np.mean(average_points, axis=0), st.sem(average_points), st.t.ppf((1+confidence)/2., trials-1)
    lower, upper = mean - m*sem, mean + m*sem
    rounded_mean, rounded_lower, rounded_upper = [round(num, 1) for num in mean], [round(num, 1) for num in lower], [round(num, 1) for num in upper]
    return [rounded_mean, rounded_lower, rounded_upper]


run(pos_cat, 100, glob_spots, 10, 10, pos_players, test_data, opp_df, 0.8)

[[41.2, 32.6, 30.5, 29.7, 33.1, 32.3, 32.8, 29.0, 32.1, 35.6],
 [38.9, 29.2, 25.8, 26.3, 28.5, 28.4, 28.8, 25.9, 28.3, 32.3],
 [43.5, 36.0, 35.2, 33.1, 37.7, 36.2, 36.8, 32.1, 35.9, 38.9]]

In [134]:
games = [50,60,70,80,90,95,98]
trials = 30
num_teams = 10
discount = 0.8
confidence = 0.95

for num_games in games:
    out = run(pos_cat, num_games, glob_spots, num_teams, trials, pos_players, test_data, opp_df, discount, confidence)
    print(out)

[[34.3, 32.1, 32.5, 32.8, 33.9, 32.5, 32.7, 32.1, 33.2, 31.6], [33.9, 29.8, 30.1, 30.3, 31.4, 30.4, 30.4, 29.8, 31.2, 29.1], [34.7, 34.4, 35.0, 35.4, 36.4, 34.6, 35.0, 34.5, 35.1, 34.1]]
[[34.2, 33.5, 34.2, 30.3, 32.4, 35.0, 32.2, 31.9, 33.1, 31.8], [33.8, 31.2, 32.2, 27.8, 29.9, 32.4, 29.5, 29.6, 30.4, 29.9], [34.6, 35.7, 36.2, 32.8, 34.9, 37.5, 34.9, 34.2, 35.7, 33.8]]
[[33.3, 32.7, 33.9, 32.0, 33.7, 32.2, 31.8, 33.4, 33.1, 31.6], [32.0, 29.8, 31.4, 29.5, 31.2, 29.8, 29.4, 31.2, 30.5, 28.8], [34.6, 35.5, 36.5, 34.6, 36.1, 34.7, 34.2, 35.5, 35.7, 34.3]]
[[33.0, 32.3, 31.1, 33.3, 32.5, 32.6, 33.5, 32.7, 33.7, 32.5], [32.4, 30.0, 29.1, 31.3, 30.6, 30.7, 31.6, 30.4, 31.6, 30.5], [33.6, 34.6, 33.1, 35.3, 34.4, 34.5, 35.3, 35.0, 35.8, 34.5]]
[[40.5, 33.4, 31.9, 31.2, 32.8, 31.9, 32.8, 31.2, 30.9, 32.0], [39.8, 31.6, 30.0, 29.0, 31.3, 30.1, 31.0, 29.2, 28.9, 30.0], [41.2, 35.2, 33.7, 33.4, 34.2, 33.7, 34.7, 33.1, 32.9, 33.9]]
[[39.2, 31.6, 32.3, 31.8, 33.2, 31.4, 31.6, 33.0, 32.1, 31.7], [3

In [135]:
discounts = [0.5, 0.7, 0.8, 0.9, 0.95, 0.99, 1.0, 1.5]
for discount in discounts:
    for num_games in games:
        trial_run = run(pos_cat, num_games, glob_spots, num_teams, trials, pos_players, test_data, opp_df, discount)
        print(trial_run)
        print('Discount factor: ' + str(discount) + ', number of games: ' + str(num_games))

[[33.8, 33.0, 31.2, 34.3, 31.7, 33.1, 33.1, 34.0, 32.1, 31.3], [33.1, 30.7, 28.7, 31.8, 29.0, 30.5, 30.3, 31.3, 29.2, 28.2], [34.5, 35.4, 33.7, 36.8, 34.4, 35.8, 35.8, 36.8, 35.0, 34.4]]
Discount factor: 0.5, number of games: 50
[[33.6, 32.7, 33.3, 31.4, 32.9, 32.3, 35.2, 33.6, 30.5, 32.3], [32.5, 30.1, 30.9, 29.0, 30.4, 30.3, 32.4, 31.3, 28.1, 29.4], [34.6, 35.3, 35.6, 33.8, 35.5, 34.3, 38.0, 35.8, 32.8, 35.2]]
Discount factor: 0.5, number of games: 60
[[31.6, 32.9, 34.1, 33.5, 33.3, 32.6, 32.2, 33.3, 29.8, 33.5], [30.4, 30.3, 31.8, 31.7, 30.7, 29.5, 30.2, 30.6, 27.1, 31.0], [32.9, 35.6, 36.5, 35.3, 35.9, 35.7, 34.2, 36.0, 32.6, 36.0]]
Discount factor: 0.5, number of games: 70
[[32.8, 32.9, 33.7, 32.7, 33.4, 33.8, 29.5, 32.7, 32.5, 32.7], [32.0, 31.1, 31.5, 30.7, 31.2, 31.5, 26.5, 30.6, 30.4, 31.0], [33.6, 34.7, 35.9, 34.6, 35.7, 36.1, 32.4, 34.7, 34.6, 34.3]]
Discount factor: 0.5, number of games: 80
[[39.9, 33.0, 32.3, 31.6, 31.5, 32.1, 32.1, 33.2, 31.2, 31.4], [39.2, 31.2, 30.6, 29

[[32.1, 30.6, 35.0, 31.8, 32.2, 34.1, 33.7, 34.5, 31.3, 32.9], [31.8, 28.3, 32.9, 29.6, 29.8, 32.1, 31.4, 32.5, 29.2, 30.8], [32.4, 32.9, 37.2, 34.0, 34.6, 36.1, 35.9, 36.5, 33.4, 35.0]]
Discount factor: 0.99, number of games: 60
[[31.0, 32.2, 33.7, 34.0, 33.9, 31.5, 31.3, 34.6, 32.7, 34.0], [30.5, 30.1, 31.4, 31.7, 31.3, 29.3, 28.8, 32.3, 30.3, 32.2], [31.5, 34.4, 36.0, 36.4, 36.4, 33.6, 33.8, 36.9, 35.1, 35.9]]
Discount factor: 0.99, number of games: 70
[[30.2, 31.5, 33.9, 32.8, 34.2, 33.0, 31.9, 33.0, 32.9, 34.1], [29.4, 29.7, 32.1, 31.0, 32.3, 31.1, 30.2, 31.4, 31.6, 32.1], [31.0, 33.3, 35.7, 34.7, 36.0, 35.0, 33.6, 34.5, 34.2, 36.1]]
Discount factor: 0.99, number of games: 80
[[37.1, 31.6, 34.1, 29.3, 31.4, 32.8, 31.8, 32.5, 34.2, 32.8], [36.3, 30.1, 32.5, 27.7, 29.7, 30.9, 29.8, 30.5, 32.6, 31.0], [37.8, 33.1, 35.8, 31.0, 33.0, 34.7, 33.8, 34.4, 35.8, 34.6]]
Discount factor: 0.99, number of games: 90
[[35.1, 34.5, 32.1, 32.0, 34.3, 31.7, 32.7, 32.0, 31.5, 32.8], [34.0, 32.5, 30.1

In [ ]:
for discount in discounts:
    total_trials_per_dis = []
    print('Discount factor: ' + str(discount))
    for num_games in games:
        trial_run = run(pos_cat, num_games, glob_spots, num_teams, trials, pos_players, test_data, opp_df, discount)
        print(trial_run)
        print('Number of games: ' + str(num_games))
        total_trials_per_dis.append(trial_run)
    avg_score_per_discount = np.mean(total_trials_per_dis, axis=0)
    rounded_scores = [round(num, 1) for num in avg_score_per_discount]
    print ('Average score for discount: ' + str(rounded_scores))

In [137]:
trial_discounts = [0.7, 0.8, 0.95, 1.0]
trials = 100
num_games = 98
confidence = 0.99

for discount in trial_discounts:
    print('Discount factor: ' + str(discount))
    trial_run = run(pos_cat, num_games, glob_spots, num_teams, trials, pos_players, test_data, opp_df, discount, confidence)
    print(trial_run)

Discount factor: 0.7
[[42.1, 33.4, 32.3, 32.2, 32.2, 30.8, 31.4, 32.2, 30.6, 31.2], [41.3, 31.5, 30.5, 30.4, 30.4, 29.0, 29.5, 30.3, 28.6, 29.3], [43.0, 35.4, 34.1, 33.9, 34.1, 32.7, 33.3, 34.1, 32.5, 33.2]]
Discount factor: 0.8
[[42.1, 32.6, 30.2, 31.9, 32.2, 32.3, 31.3, 31.2, 33.0, 32.0], [41.3, 30.6, 28.2, 30.1, 30.3, 30.4, 29.5, 29.1, 31.2, 30.1], [43.0, 34.6, 32.1, 33.7, 34.0, 34.1, 33.2, 33.2, 34.8, 33.9]]
Discount factor: 0.95
[[39.7, 32.2, 32.6, 33.0, 31.6, 32.3, 31.4, 31.7, 32.2, 31.8], [39.1, 30.4, 30.8, 31.4, 29.9, 30.7, 29.6, 30.0, 30.6, 29.9], [40.3, 33.9, 34.4, 34.6, 33.3, 34.0, 33.3, 33.4, 33.9, 33.6]]
Discount factor: 1.0
[[38.7, 32.2, 32.8, 32.0, 32.1, 32.9, 32.6, 32.2, 31.6, 31.9], [38.2, 30.8, 30.9, 30.6, 30.5, 31.5, 30.7, 30.7, 30.1, 30.1], [39.2, 33.5, 34.6, 33.4, 33.7, 34.3, 34.4, 33.8, 33.2, 33.7]]
